In [1]:
import sys
import numpy as np
import pandas as pd
from utils import *
sys.path.append('../')
from bayes_ensemble.bayes_ensemble import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_preds_all.head()

,date,pred,lower,upper,state,model_id
0,2022-10-09,110.690113,64.291013,219.826948,AC,21
1,2022-10-16,144.952846,77.564037,279.722708,AC,21
2,2022-10-23,163.613937,92.915072,276.591399,AC,21
3,2022-10-30,170.240443,87.197631,378.407811,AC,21
4,2022-11-06,215.117888,111.835667,445.024983,AC,21


### Applying the base model

In [3]:
%%time 
df_for_val_base = pd.DataFrame()

df_23_24 = pd.DataFrame()

df_25 = pd.DataFrame()

df_w = pd.DataFrame()

dist = 'log_normal'
for state in states_BR: 
    
    data_23, data_24, preds_23, preds_24, preds_25, models = format_data(state, models_by_state, data_all, df_preds_all)

    K = len(models)
    weights = np.full(K, 1/(K))
    
    ens_23 = Ensemble(df= preds_23,
            order_models = models, dist = dist)
    
    df_23_in = ens_23.apply_ensemble(weights = weights)

    ens_24 = Ensemble(df= preds_24,
            order_models = models,  dist = dist)

    df_24_out = ens_24.apply_ensemble(weights = weights)

    df_23_24_ = pd.concat([df_23_in, df_24_out], ignore_index = True)

    df_23_24_['state'] = state
 
    df_23_24 = pd.concat([df_23_24, df_23_24_], ignore_index = True)
    

CPU times: user 14min 13s, sys: 3.48 s, total: 14min 16s
Wall time: 14min 44s


In [4]:
df_23_24.to_csv(f'../predictions/ensemble_2023_2024_base_{dist}.csv', index = False)
#df_25.to_csv(f'../predictions/ensemble_2025_base_{dist}.csv', index = False)